In [10]:
library(data.table) 
library(StanHeaders)
library(prophet)
library(rstan)
library(ggplot2)
library(reticulate)
library(Robyn)

# Training MMM Model Using Facebook's Robyn

## Daily Model

In [42]:
dt_data = fread("../data/processed/search_df_select.csv")
head(dt_data, 2)

revenue,google_search_Bottom,google_search_Mid,google_search_Top,bing_search_Bottom,bing_search_Top,facebook_Bottom,facebook_Mid,facebook_Top,google,bing,fb,top,mid,bottom,date
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<date>
30638.84,248.73,0.27,1286.40,17.23,49.94,121.58,220.94,5416.81,3070.80,134.34,5759.33,13506.30,442.42,775.08,2019-01-01
30134.13,209.05,1.94,1243.62,18.18,59.97,120.67,216.65,5711.90,2909.22,156.30,6049.22,14030.98,437.18,695.80,2019-01-02
31172.54,232.55,4.58,1349.84,18.48,52.71,119.48,220.88,7191.77,3173.94,142.38,7532.13,17188.64,450.92,741.02,2019-01-03
22587.84,289.16,7.50,1229.11,23.32,58.14,118.49,197.02,7005.19,3051.54,162.92,7320.70,16584.88,409.04,861.94,2019-01-04
26166.98,281.36,2.00,1422.20,18.90,46.91,121.03,196.41,5057.96,3411.12,131.62,5375.40,13054.14,396.82,842.58,2019-01-05
32981.04,286.91,65.37,1738.27,24.53,59.58,120.51,196.31,5010.45,4181.10,168.22,5327.27,13616.60,523.36,863.90,2019-01-06


In [15]:
robyn_object <- "Robyn.RData"

In [38]:
InputCollect <- robyn_inputs(
dt_input = fread("../data/processed/search_df_select.csv")
,dt_holidays = dt_prophet_holidays
,date_var = "date"
,dep_var = "revenue"
,dep_var_type = "revenue"
,prophet_vars = c("trend", "season", "weekday")
,prophet_signs = c("default","default", "default")
,prophet_country ='US'
,paid_media_vars = c("google_search_Bottom", "google_search_Mid", "google_search_Top", "bing_search_Bottom", "bing_search_Top", "facebook_Bottom", "facebook_Mid", "facebook_Top")
,paid_media_signs = c("positive", "positive","positive", "positive", "positive","positive", "positive","positive")
,paid_media_spends = c("google_search_Bottom", "google_search_Mid", "google_search_Top", "bing_search_Bottom", "bing_search_Top", "facebook_Bottom", "facebook_Mid", "facebook_Top")

# parameters
,cores = 3
,window_start = "2019-01-01"
,window_end = "2021-07-31"

#core model params
,adstock = "geometric"
,iterations = 500
,nevergrad_algo = "TwoPointsDE"
,trials = 5
)
    

'hyperparameters' are not provided yet. To include them, run robyn_inputs(InputCollect = InputCollect, hyperparameters = ...)



In [39]:
#hyperparameters

#plot_adstock(plot = FALSE)
#plot_saturation(plot = FALSE)

hyperparameters <- list(
  google_search_Bottom_alphas = c(0.5, 3)
  ,google_search_Bottom_gammas = c(0.3, 1)
  ,google_search_Bottom_thetas = c(0, 0.3)
    
  ,google_search_Mid_alphas = c(0.5, 3)
  ,google_search_Mid_gammas = c(0.3, 1)
  ,google_search_Mid_thetas = c(0, 0.3)

 ,google_search_Top_alphas = c(0.5, 3)
  ,google_search_Top_gammas = c(0.3, 1)
  ,google_search_Top_thetas = c(0, 0.3)

  ,bing_search_Bottom_alphas = c(0.5, 3)
  ,bing_search_Bottom_gammas = c(0.3, 1)
  ,bing_search_Bottom_thetas = c(0, 0.3)
    
  ,bing_search_Top_alphas = c(0.5, 3)
  ,bing_search_Top_gammas = c(0.3, 1)
  ,bing_search_Top_thetas = c(0, 0.3)  
    
  ,facebook_Bottom_alphas = c(0.5, 3)
  ,facebook_Bottom_gammas = c(0.3, 1)
  ,facebook_Bottom_thetas = c(0, 0.3)
    
  ,facebook_Mid_alphas = c(0.5, 3)
  ,facebook_Mid_gammas = c(0.3, 1)
  ,facebook_Mid_thetas = c(0, 0.3)
    
  ,facebook_Top_alphas = c(0.5, 3)
  ,facebook_Top_gammas = c(0.3, 1)
  ,facebook_Top_thetas = c(0, 0.3)
    )


In [40]:
InputCollect <- robyn_inputs(InputCollect = InputCollect, hyperparameters = hyperparameters)

Warning message in check_iteration(InputCollect$calibration_input, InputCollect$iterations, :
“We recommend to run at least 2000 iterations per trial and 5 trials to build initial model”


In [41]:
OutputCollect <- robyn_run(
  InputCollect = InputCollect # feed in all model specification
  , plot_folder = robyn_object # plots will be saved in the same folder as robyn_object
  , pareto_fronts = 3
  , plot_pareto = TRUE
  # , calibration_constraint = 0.1 # run ?robyn_run to see description
  # , lambda_control = 1 # run ?robyn_run to see description
  )

Provided 'plot_folder' doesn't exist. Using default 'plot_folder = getwd()': /Users/Jaime/repos/search-for-green/notebooks

Input data has 943 days in total: 2019-01-01 to 2021-07-31

Initial model is built on rolling window of 943 days: 2019-01-01 to 2021-07-31

Using geometric adstocking with 24 hyperparameters & 10-fold ridge x-validation on 3 cores

>>> Start running 5 trials with 500 iterations per trial each with TwoPointsDE nevergrad algorithm...

 Running trial nr. 1



  |======================================================================| 100%


 Finished in 2.99 mins



 Running trial nr. 2



  |======================================================================| 100%

...

  |======================================================================| 100%


 Finished in 3.89 mins



>>> Collecting results...

>>> Exporting all charts into directory: /Users/Jaime/repos/search-for-green/notebooks/2021-11-09 00.55 init...

>>> Plotting summary charts...

No spend-exposure modelling needed. all media variables used for mmm are spend variables 

>>> Plotting 57 Pareto optimum models...



  |======================================================================| 100%


Total time: 19.96 mins



In [43]:
OutputCollect$allSolutions # get all model IDs in result
select_model <- "2_118_2" # select one from above
robyn_save(robyn_object = robyn_object # model object location and name
           , select_model = select_model # selected model ID
           , InputCollect = InputCollect # all model input
           , OutputCollect = OutputCollect # all model output
)

[1] "2_118_2" "2_122_2" "2_135_2" "2_140_1" "2_155_2" "2_80_3"  "3_158_2"
 [8] "4_141_3" "4_153_3" "4_154_2" "5_115_2" "5_145_2" "5_151_3" "5_158_3"
[15] "5_164_1" "5_166_3" "2_145_2" "2_60_3"  "2_90_2"  "3_125_1" "3_165_3"
[22] "4_157_3" "4_166_2" "5_101_2" "5_102_2" "5_112_2" "5_132_2" "5_141_3"
[29] "5_160_1" "5_162_1" "5_162_2" "5_79_3"  "5_82_2"  "2_107_1" "2_115_2"
[36] "2_137_1" "2_54_2"  "2_85_1"  "3_148_2" "3_164_2" "4_151_3" "4_153_2"
[43] "4_157_2" "4_161_3" "4_163_3" "4_165_2" "5_111_2" "5_119_3" "5_122_1"
[50] "5_122_2" "5_123_3" "5_124_2" "5_136_1" "5_154_1" "5_158_2" "5_165_2"
[57] "5_167_1"

Robyn.RData already exists. Are you certain to overwrite it? (Yes/no/cancel) Yes


![Robyn Daily Model](../assets/2_118_2.png)

## Weekly Model

In [62]:
robyn_object <- "robyn_weekly/Robyn_weekly.RData"

In [63]:
InputCollect <- robyn_inputs(
dt_input = fread("../data/processed/weekly.csv")
,dt_holidays = dt_prophet_holidays
,date_var = "date"
,dep_var = "revenue"
,dep_var_type = "revenue"
,prophet_vars = c("trend", "season")
,prophet_signs = c("default","default")
,prophet_country ='US'
,paid_media_vars = c("google_search_Bottom", "google_search_Mid", "google_search_Top", "bing_search_Bottom", "bing_search_Top", "facebook_Bottom", "facebook_Mid", "facebook_Top")
,paid_media_signs = c("positive", "positive","positive", "positive", "positive","positive", "positive","positive")
,paid_media_spends = c("google_search_Bottom", "google_search_Mid", "google_search_Top", "bing_search_Bottom", "bing_search_Top", "facebook_Bottom", "facebook_Mid", "facebook_Top")

# parameters
,cores = 3
,window_start = "2019-01-07"
,window_end = "2021-08-02"

#core model params
,adstock = "geometric"
,iterations = 500
,nevergrad_algo = "TwoPointsDE"
,trials = 5
)
    

'hyperparameters' are not provided yet. To include them, run robyn_inputs(InputCollect = InputCollect, hyperparameters = ...)



In [64]:
#hyperparameters

#plot_adstock(plot = FALSE)
#plot_saturation(plot = FALSE)

hyperparameters <- list(
  google_search_Bottom_alphas = c(0.5, 3)
  ,google_search_Bottom_gammas = c(0.3, 1)
  ,google_search_Bottom_thetas = c(0, 0.3)
    
  ,google_search_Mid_alphas = c(0.5, 3)
  ,google_search_Mid_gammas = c(0.3, 1)
  ,google_search_Mid_thetas = c(0, 0.3)

 ,google_search_Top_alphas = c(0.5, 3)
  ,google_search_Top_gammas = c(0.3, 1)
  ,google_search_Top_thetas = c(0, 0.3)

  ,bing_search_Bottom_alphas = c(0.5, 3)
  ,bing_search_Bottom_gammas = c(0.3, 1)
  ,bing_search_Bottom_thetas = c(0, 0.3)
    
  ,bing_search_Top_alphas = c(0.5, 3)
  ,bing_search_Top_gammas = c(0.3, 1)
  ,bing_search_Top_thetas = c(0, 0.3)  
    
  ,facebook_Bottom_alphas = c(0.5, 3)
  ,facebook_Bottom_gammas = c(0.3, 1)
  ,facebook_Bottom_thetas = c(0, 0.3)
    
  ,facebook_Mid_alphas = c(0.5, 3)
  ,facebook_Mid_gammas = c(0.3, 1)
  ,facebook_Mid_thetas = c(0, 0.3)
    
  ,facebook_Top_alphas = c(0.5, 3)
  ,facebook_Top_gammas = c(0.3, 1)
  ,facebook_Top_thetas = c(0, 0.3)
    )


In [65]:
InputCollect <- robyn_inputs(InputCollect = InputCollect, hyperparameters = hyperparameters)

Warning message in check_iteration(InputCollect$calibration_input, InputCollect$iterations, :
“We recommend to run at least 2000 iterations per trial and 5 trials to build initial model”


In [66]:
OutputCollect <- robyn_run(
  InputCollect = InputCollect # feed in all model specification
  , plot_folder = robyn_object # plots will be saved in the same folder as robyn_object
  , pareto_fronts = 3
  , plot_pareto = TRUE
  # , calibration_constraint = 0.1 # run ?robyn_run to see description
  # , lambda_control = 1 # run ?robyn_run to see description
  )

Provided 'plot_folder' doesn't exist. Using default 'plot_folder = getwd()': /Users/Jaime/repos/search-for-green/notebooks

Input data has 135 weeks in total: 2019-01-07 to 2021-08-02

Initial model is built on rolling window of 135 weeks: 2019-01-07 to 2021-08-02

Using geometric adstocking with 24 hyperparameters & 10-fold ridge x-validation on 3 cores

>>> Start running 5 trials with 500 iterations per trial each with TwoPointsDE nevergrad algorithm...

 Running trial nr. 1



  |======================================================================| 100%


 Finished in 3.17 mins



 Running trial nr. 2



  |======================================================================| 100%

...

  |======================================================================| 100%


 Finished in 3.52 mins



>>> Collecting results...

>>> Exporting all charts into directory: /Users/Jaime/repos/search-for-green/notebooks/2021-11-09 04.02 init...

>>> Plotting summary charts...

No spend-exposure modelling needed. all media variables used for mmm are spend variables 

>>> Plotting 43 Pareto optimum models...



  |======================================================================| 100%


Total time: 20.24 mins



In [68]:
# Save model 3_165_2
OutputCollect$allSolutions # get all model IDs in result
select_model <- "3_165_2" # select one from above
robyn_save(robyn_object = robyn_object # model object location and name
           , select_model = select_model # selected model ID
           , InputCollect = InputCollect # all model input
           , OutputCollect = OutputCollect # all model output
)

[1] "2_117_1" "2_145_1" "2_154_3" "3_149_2" "3_155_2" "3_162_3" "3_165_2"
 [8] "3_76_3"  "3_96_1"  "3_99_3"  "4_135_1" "2_134_1" "2_134_3" "2_95_1" 
[15] "3_129_3" "3_135_2" "3_137_1" "3_141_2" "3_154_1" "3_156_1" "3_163_1"
[22] "3_163_3" "3_165_1" "3_79_3"  "3_95_2"  "4_133_1" "4_55_3"  "5_127_2"
[29] "2_114_1" "2_144_1" "3_127_3" "3_144_1" "3_148_3" "3_163_2" "3_69_3" 
[36] "4_118_3" "4_123_1" "4_138_3" "4_147_3" "4_152_3" "4_79_3"  "5_101_3"
[43] "5_111_3"

![Robyn Weekly Model](../assets/3_165_2.png)